<!-- Autoheader begin -->
<hr/>
<div id="navtitle_2_2_py" style="text-align:center; font-size:16px">II.2 Parameter Optimization for STIRAP</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_1_TLS.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_2_1_TLS.ipynb" style="font-size:13px">II.1 Population Inversion in a Two-Level-System using Parameter Optimization</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_1_2_lambda.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="py_exercise_1_2_lambda.ipynb" style="font-size:13px">I.2 Population Transfer in a Three-Level-System with STIRAP</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_3_chiral.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_2_3_chiral.ipynb" style="font-size:13px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_2_lambda.ipynb" style="font-size:13px">III.2 Optimal Control for STIRAP</a><br>
        <a href="py_exercise_3_2_lambda.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_2_2_lambda.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Parameter Optimization for STIRAP

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid
#2\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}$

In this notebook we return to the three-level-system in a Lambda configuration introduced in [Exercise I.2](py_exercise_1_2_lambda.ipynb). Our goal remains to achieve population transform from level 1 to level 3 without populating the intermediate level 2. In this notebook you will learn how to use gradient-free parameter optimization for this purpose. A particularly important part of the optimization is the definition of an appropriate optimization functional which incorporates the goal to avoid populating level 2 as an additional condition.

## Setup

In [ ]:
import qutip as qt  # QUantum Toolbox In Python
import numpy as np  # package for numerical functions such as cos, sin, etc.
import matplotlib.pylab as plt
import nlopt
import functools
from scipy.integrate import complex_ode

# Some utilities for showing hints and solutions
from utils.exercise_2_lambda import *

## Model

As a reminder, the STIRAP Hamiltonian is given by the following expression

\begin{align} H_{\text{STIRAP}}=\begin{pmatrix}0 & \frac{1}{2}\Omega^{*}_{\text{P}}(t)\\
\frac{1}{2}\Omega_{\text{P}}(t) & \Delta_\text{P} & \frac{1}{2}\Omega^{*}_{\text{S}}(t)\\
& \frac{1}{2}\Omega_{\text{S}}(t) & \Delta_\text{P}-\Delta_\text{S} \end{pmatrix}
\end{align}

and gives rise to a level scheme in a Lambda configuration

<center><img src="../figures/lambda_system_levels.png" alt="Lambda system considered in this notebook" width="500"></center>

We now use parameter optimization to find the right pulse shapes, assuming once again a Gaussian pulse shape for simplicity. Furthermore we assume to be on two-photon resonance and fix the single-photon detuning at $\Delta \equiv 
\Delta_\text{P} = 1$ which defines a reference energy. Therefore there are 3 parameters for each pulse which can be individually tuned: Its temporal position, its temporal width and its strength resulting. All in all, we have a total of 6 parameters.

Our functional will thus take a list of these 6 parameters as an input to calculate how close we are to the following two goals:

 1. At final time, all population should be in state 3.
 2. Throughout the evolution, population in 2 should be kept as close to zero as possible.

A straightforward approach to construct functionals which take multiple physical goals into account is to simply sum up functionals for each of the individual goals. In our case this is achieved with the following definition,

\begin{align}
    \mathcal{F} &= \Braket{3}{\psi (t_f)} - \frac{1}{T} \int_{t_i}^{t_f} \Braket{2}{\psi(t)} dt \,,
\end{align}

where $t_i$, $t_f$ is the initial resp. final time and $T=t_f - t_i$ is the
total duration of the protocol. This fidelity has a maximum value of $1$ corresponding to both goals being achieved perfectly, i.e., the final state of the evolution is $\ket{3}$ and there is no poulation in $\ket{2}$ during the
entire protocol. Since we frame our optimizations as minimizations, we thus attempt to minimize the functional $1 - \mathcal{F}$ in the following.

The optimization functional will in the following be encoded by the function `f_stirap` which can be passed to the
non-linear optimization library `nlopt`.

## Shape functions

First we define the pulse shape functions and the Hamiltonian.

In [ ]:
def gaussian_shape(t, kwargs):
    """
    Gaussian shape function centered around kwargs["t_0"] with width kwargs["\sigma"]^2
    """
    return np.exp(-1/2*(t - kwargs["t_0"])**2/kwargs["σ"]**2) / np.sqrt(2*np.pi*kwargs["σ"])

In [ ]:
def pump_shape(t, kwargs):
    """
    Shape function for the pump pulse
    """
    return gaussian_shape(t, {"t_0": kwargs["t_p"], "σ": kwargs["σ_p"]})

In [ ]:
def stokes_shape(t, kwargs):
    """
    Shape function for the Stokes pulse
    """
    return gaussian_shape(t, {"t_0": kwargs["t_s"], "σ": kwargs["σ_s"]})

In [ ]:
def h_stirap(t, kwargs, qutip=True):
    """
    Function returning the STIRAP Hamiltonian as a QuTiP object
    """
    Ω_p = pump_shape(t, kwargs)
    Ω_s = stokes_shape(t, kwargs)

    out = np.zeros(shape=(3,3), dtype=complex)
    out[1,1] = kwargs["Δ"]

    out[0,1] = kwargs["d_12"] * Ω_p / 2
    out[1,0] = np.conj(out[0,1])

    out[1,2] = kwargs["d_23"] * Ω_s / 2
    out[2,1] = np.conj(out[1,2])

    if qutip:
        return qt.Qobj(out)
    return out

## Parameter optimization

Now we can define the optimization functional. To speed up the optimization we will not use the solver provided by `QuTip` but rather the more efficient `scipy.integrate.complex_ode`.

In [ ]:
def schroedinger_ode(t, y, kwargs):
    """
    The Schroedinger differential equation
    """
    h = h_stirap(t, kwargs, qutip=False)
    return -1j*h @ y

def f_stirap(x, grad=None):
    """
    Functional for STIRAP parameter optimization
    """
    global iterations
    global obtained_functional_values
    iterations += 1
    kwargs = {
    "t_p": x[0], # Center of the Pump pulse
    "t_s": x[1], # Center of the Stokes pulse

    "σ_p": x[2], # Width of the Pump pulse
    "σ_s": x[3], # Width of the Stokes pulse

    "Δ": 1, # Single-photon detuning
    "d_12": x[4], # Electric-dipole moment for 1-2 transition / Pump Strength
    "d_23": x[5], # Electric-dipole moment for 2-3 transition / Stokes Strength
    }

    schroedinger_ode_with_args = functools.partial(schroedinger_ode, kwargs=kwargs)
    ode_solver = complex_ode(schroedinger_ode_with_args)
    ode_solver = ode_solver.set_initial_value(np.array(psi_0)[:,0], t_list.min())

    y_sol, t_sol = [ode_solver.y], [ode_solver.t]
    while ode_solver.successful() and ode_solver.t < t_list.max():
        ode_solver.integrate(ode_solver.t+dt)
        y_sol.append(ode_solver.y)
        t_sol.append(ode_solver.t)
    y_sol, t_sol = np.array(y_sol), np.array(t_sol)

    functional_value = 1 - (abs(y_sol[-1,-1])**2 - np.sum(np.abs(y_sol[:,1])**2)/len(t_sol))

    obtained_values.append(functional_value)

    print(f"Iteration: {iterations:}, current functional value {functional_value:8.4f}", end="\r")
    return functional_value

t_list, dt = np.linspace(-400, 400, 100, retstep=True) # time-interval

p1, p2, p3 = qt.projection(3,0,0), qt.projection(3,1,1), qt.projection(3,2,2)

psi_0 = qt.basis(3, 0) # initial state
print("Initial state: ")
display(psi_0)

### Problem 1: Choosing a guess pulse and running the optimization

In order for the optimization to be successful, we need to specify bounds for
the optimization parameters. Moreover, the result of the optimization heavily
depends on the initial guess parameters. Try finding a set of initial
parameters that will converge to 99% fidelity!

In [ ]:
# define lower and upper bounds for three pulse durations, three phis and three
# E0s that are to be optimized
bounds_lower = [-200,-200, 10,10, 0,0]
bounds_upper = [200,200, 80,80, 100,100]

## determine guess parameters for all parameters that are optimized
# set guess parameters for all parameters that are optimized
# The order is [t_p, t_s, σ_p, σ_s, d_12, d_23]
guess=[insert_guess_parameters_here]

# specify the optimization method and the number of parameters that should be optimized
# (given here by len(bounds_lower))
opt = nlopt.opt(nlopt.LN_NELDERMEAD, len(bounds_lower))

# set the lower and upper bound for the optimization
opt.set_lower_bounds(bounds_lower)
opt.set_upper_bounds(bounds_upper)
opt.set_maxeval(200)

# set the objective that should be optimized (note that this implies minimisation)
opt.set_min_objective(f_stirap)

# fidelity above which the optimization should be stopped
fid_min = 0.99

# define below which value, i.e., error, to stop the optimization
opt.set_stopval(1.-fid_min)

# perform the optimization
obtained_values = []
iterations = 0
x = opt.optimize(guess)

print('\n\nHighest fidelity reached: {:6.2f}%'.format(100.0*(1.-opt.last_optimum_value())))
if opt.last_optimum_value() > 1-fid_min:
    print('\tbad guess, please try again!')
else:
    print('\tcongratulations, you have found a perfect STIRAP protocol!')

In [ ]:
#problem_1.hint
#problem_1.solution

## Analyze optimization results

Finally, let's have a look at the optimized results:

In [ ]:
kwargs = {
    "t_p": x[0], # Center of the Pump pulse
    "t_s": x[1], # Center of the Stokes pulse

    "σ_p": x[2], # Width of the Pump pulse
    "σ_s": x[3], # Width of the Stokes pulse

    "Δ": 1, # Single-photon detuning
    "d_12": x[4], # Electric-dipole moment for 1-2 transition / Pump Strength
    "d_23": x[5], # Electric-dipole moment for 2-3 transition / Stokes Strength
    }

se_result = qt.sesolve(h_stirap, psi_0, t_list, args=kwargs, e_ops=[p1,p2,p3])

fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].plot(t_list, pump_shape(t_list, kwargs), label="pump")
ax[0].plot(t_list, stokes_shape(t_list, kwargs), label="stokes")
ax[0].set_xlabel("time")
ax[0].set_ylabel(r"$\Omega(t)$")
ax[0].legend()

for i_state, population in enumerate(se_result.expect):
    ax[1].plot(t_list, population, label=f"level {i_state+1}")
ax[1].legend()
ax[1].set_xlabel("time")
ax[1].set_ylabel("Populations")
plt.show()

## Next steps

To go another step up in system complexity with parameter optimization we recommend [Exercise II.3](py_exercise_2_3_chiral.ipynb) which discusses the very interesting case of three-wave-mixing in a chiral molecule modelled by a three-level system. Alternatively, if you are interested in optimization with a gradient-based approach, we recommend to have a look at [Exercise III.2](py_exercise_3_2_lambda.ipynb)  in which Krotov's method is used for the opimization you studied in this notebook.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_2_2_py)
<!-- Autofooter end -->